<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Preprocess_images_for_DreamBooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**DreamBooth**

768pix x 768pixに成形する必要がある

甲状腺眼症画像： olympiaのtreatedデータセット + Handai super extend dataset (3000枚ぐらい)
コントロール画像： Handai super extend dataset


#**オリンピアデータセットの抽出**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import shutil
import glob

full_dataset_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset" #編集未の画像を格納されているフォルダ
reference_path = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_500px" #治療された患者のIDが格納されているフォルダ

print(len(glob.glob(f"{reference_path}/*")))

# reference_path内のファイルのベース名を取得
reference_files = [os.path.basename(file) for file in glob.glob(os.path.join(reference_path, "*"))]

# full_dataset_path内のファイルと比較して一致するファイルのパスを取得
matching_files = [file for file in glob.glob(os.path.join(full_dataset_path, "*")) if os.path.basename(file) in reference_files]

# 最初の5つのファイルパスを表示
for file_path in matching_files[:5]:
    print(file_path)


561
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/19.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/28.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/30.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/35.JPG
/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/36.JPG


In [5]:
#img2square
import os
import shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from PIL import Image

def convert(matching_files, out_path):
    start = time.time()

    for i in matching_files:
          img = Image.open(i)
          # "olympia_"をファイル名の先頭に追加
          new_filename = "olympia_" + os.path.basename(i)
          img_new = expand2square(img, (0, 0, 0)).resize((768, 768))
          img_new.save(os.path.join(out_path, new_filename))
          print(os.path.join(out_path, new_filename))

    print('Process done!!')
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result


In [6]:
#画像出力
out_path = '/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended'
if not os.path.exists(out_path):
    os.makedirs(out_path)

convert(matching_files, out_path)

/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_19.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_28.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_30.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_35.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_36.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_38.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_42.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_43.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_41.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_45.JPG
/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended/olympia_49.JPG

#**Handai Super extend datasetからの抽出**

In [7]:
grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

def count_files(directory):
    file_count = 0
    for _, _, files in os.walk(directory):
        file_count += len(files)

    return file_count

grav_file_count = count_files(grav_dir)
cont_file_count = count_files(cont_dir)

print(f"Number of files in '{grav_dir}': {grav_file_count}")
print(f"Number of files in '{cont_dir}': {cont_file_count}")

Number of files in '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav': 2340
Number of files in '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont': 1955


In [8]:
def convert(in_path, out_path):
    #処理時間の計測
    start = time.time()

    l=0
    processing_file = showInfo(in_path)
    for i in processing_file:
          img = Image.open(in_path + '/' + i)
          img_new = expand2square(img, (0, 0, 0)).resize((768, 768))
          img_new.save(out_path +'/'+ i)
          print(out_path +'/'+ i)

          #切り取った画像を表示
          #plt.imshow(np.asarray(img_new))
          #plt.show()

    print('Process done!!')
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, (0, (height - width) // 2))
        return result

def showInfo(in_path):
    #処理するDirectoryの設定
    file = os.listdir(in_path)
    print(len(file))

    #ここにフォルダ番号を記載する (ex. [0:999])
    processing_file = file[0:]
    print(processing_file)
    len(processing_file)
    return processing_file

In [ ]:
#元画像フォルダ
in_path = grav_dir

#保存先フォルダ
out_path = "/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/grav_extended"
if not os.path.exists(out_path):
    os.makedirs(out_path)

#showInfo(in_path)
convert(in_path, out_path)

#元画像フォルダ
in_path = cont_dir

#保存先フォルダ
out_path = "/content/drive/MyDrive/Deep_learning/DreamBooth/GravCont768px/cont_extended"
if not os.path.exists(out_path):
    os.makedirs(out_path)

#showInfo(in_path)
convert(in_path, out_path)